# We start off by importing the data

In [1]:
import math
import pandas as pd
import shared
pd.reset_option('precision')
#pd.set_option('float_format', '{:f}'.format)
pd.set_option('float_format', '{:,.1f}'.format)

bl_kurzel = shared.bl_kurzel

dfs = {}
for kurzel in shared.bl_kurzel:
    dfs[kurzel] = pd.read_csv(f'data/vac_{kurzel}.csv', sep=',', index_col=0, parse_dates=True)
    # it's important to define index and parse_dates



In [2]:
list(dfs['BY'].columns)

['publication_date',
 'dosen_kumulativ',
 'personen_erst_kumulativ',
 'personen_voll_kumulativ',
 'personen_min1_kumulativ',
 'personen_zweit_kumulativ',
 'personen_auffr_kumulativ',
 'dosen_biontech_kumulativ',
 'personen_erst_biontech_kumulativ',
 'personen_voll_biontech_kumulativ',
 'personen_min1_biontech_kumulativ',
 'personen_zweit_biontech_kumulativ',
 'personen_auffr_biontech_kumulativ',
 'dosen_moderna_kumulativ',
 'personen_erst_moderna_kumulativ',
 'personen_voll_moderna_kumulativ',
 'personen_min1_moderna_kumulativ',
 'personen_zweit_moderna_kumulativ',
 'personen_auffr_moderna_kumulativ',
 'dosen_astrazeneca_kumulativ',
 'personen_erst_astrazeneca_kumulativ',
 'personen_voll_astrazeneca_kumulativ',
 'personen_min1_astrazeneca_kumulativ',
 'personen_zweit_astrazeneca_kumulativ',
 'personen_auffr_astrazeneca_kumulativ',
 'dosen_janssen_kumulativ',
 'personen_erst_janssen_kumulativ',
 'personen_voll_janssen_kumulativ',
 'personen_min1_janssen_kumulativ',
 'personen_zweit_jans

### Let' pick the few columns that interest us

In [3]:
for kurzel in bl_kurzel:
    dfs[kurzel] = dfs[kurzel][['publication_date', 'dosen_kumulativ', 'personen_erst_kumulativ', 'personen_voll_kumulativ']]

# Let's fix the data

### There are two spikes, those are just wrong data, let's fix them manually

In [4]:
dfs['BY'].dosen_kumulativ = dfs['BY'].dosen_kumulativ.replace(384311, 343668)
dfs['BY'].dosen_kumulativ = dfs['BY'].dosen_kumulativ.replace(1150129, 1186228)




### New column shots_today instead of total doses til now (dosen_kumulativ)

Each row is seen as it's own entry, the value of a day should not depend on the previous value.
But that is the case for the dosen_kumulativ column.
Instead we will add a new column 'heutige_dosen' and recreate the dosen_kumulativ column

In [5]:
dfs['BY'].head(5)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ
date,,,,
2020-12-27,2020-12-28 16:15,3389,3389,0
2020-12-28,2020-12-29 08:00,5219,5219,0
2020-12-30,2020-12-31 08:30,28206,28206,0
2020-12-31,2021-01-01 12:30,37955,37955,0
2021-01-01,2021-01-02 08:00,39005,39005,0


In [6]:
def add_dif_column(df):
    df['shots_today'] = df.dosen_kumulativ - df.dosen_kumulativ.shift(1)
    df['shots_today'] = df['shots_today'].fillna(0)
    return df.astype({'shots_today': 'int64'})

for kurzel in bl_kurzel:
    dfs[kurzel] = add_dif_column(dfs[kurzel])

### Add missing rows with empty values

In [7]:
dfs['DE'].head(10)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today
date,,,,,
2020-12-27,2020-12-28 16:15,21566,21566,0,0
2020-12-28,2020-12-29 08:00,41962,41962,0,20396
2020-12-30,2020-12-31 08:30,131626,131626,0,89664
2020-12-31,2021-01-01 12:30,165575,165575,0,33949
2021-01-01,2021-01-02 08:00,188553,188553,0,22978
2021-01-02,2021-01-03 08:00,238809,238809,0,50256
2021-01-03,2021-01-04 12:00,265986,265986,0,27177
2021-01-04,2021-01-05 12:00,316962,316962,0,50976
2021-01-05,2021-01-06 11:00,367331,367331,0,50369


In [8]:
dfs['DE'].tail(10)

,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today
date,,,,,
2022-01-02,2022-01-03 08:00,149673801,58129542,59209560,910054
2022-01-03,2022-01-04 08:00,150090299,58165615,59252481,416498
2022-01-04,2022-01-05 08:00,150983426,58236287,59371059,893127
2022-01-05,2022-01-06 08:00,151807909,58297042,59483838,824483
2022-01-06,2022-01-07 08:00,152496126,58351202,59574879,688217
2022-01-09,2022-01-10 08:00,153899268,58465277,59787106,1403142
2022-01-10,2022-01-11 08:00,154550464,58516607,59883814,651196
2022-01-11,2022-01-12 08:00,155365945,58574159,60004889,815481
2022-01-12,2022-01-13 08:00,156257670,58639456,60141697,891725


In [9]:
def fix_missing_days(df):
    # fill in the dates
    idx = pd.date_range(start='2020-12-26', end=df.index.max())

    df = df.reindex(idx)

    # Create new index column because that's waay easier than having the date column be the index
    df = df.reset_index()
    df.at[0,'dosen_kumulativ'] = 0
    df.at[0,'shots_today'] = 0
    df.at[0,'personen_erst_kumulativ'] = 0
    df.at[0,'personen_voll_kumulativ'] = 0
    df.at[1,'shots_today'] =  df.iloc[1, :]['dosen_kumulativ']

    return df.rename(columns={'index': 'date'})

for kurzel in bl_kurzel:
    dfs[kurzel] = fix_missing_days(dfs[kurzel])


### Fix NaN values in shots_today

In [10]:
def fix_NaN_dosen(df, column):
    i = 0
    while i < len(df.index):
        row = df.iloc[i, :]
        if pd.isnull(row[column]):
            j = 1
            new_row = df.iloc[i+j, :]
            while pd.isnull(new_row[column]):
                j = j + 1
                new_row = df.iloc[i+j, :]
            next_valid_row = df.iloc[i+j, :]
            quotient = next_valid_row[column] / (j+1)
            df.at[i+j,column] = quotient
            for to_change in range(i, i+j):
                df.at[to_change,column] = quotient
            i = i+j
        else:
            i = i + 1

for kurzel in bl_kurzel:
    fix_NaN_dosen(dfs[kurzel], 'shots_today')



### New shots_sum column

In [11]:
def add_shots_sum(df):
    df['shots_sum'] = 0
    df['shots_sum'] = df['shots_today'].cumsum().round()
    df.shots_sum = df.shots_sum.astype(int)
    return df

for kurzel in bl_kurzel:
    dfs[kurzel] = add_shots_sum(dfs[kurzel])

In [12]:
dfs['BY'].head(6)

,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum
0,2020-12-26,NaN,0.0,0.0,0.0,0.0,0
1,2020-12-27,2020-12-28 16:15,"3,389.0","3,389.0",0.0,"3,389.0",3389
2,2020-12-28,2020-12-29 08:00,"5,219.0","5,219.0",0.0,"1,830.0",5219
3,2020-12-29,NaN,NaN,NaN,NaN,"11,493.5",16712
4,2020-12-30,2020-12-31 08:30,"28,206.0","28,206.0",0.0,"11,493.5",28206
5,2020-12-31,2021-01-01 12:30,"37,955.0","37,955.0",0.0,"9,749.0",37955


### New column: Day of the week

In [13]:
for kurzel in bl_kurzel:
    dfs[kurzel] = shared.add_weekday_stuff(dfs[kurzel], 'date')

## Add columns 'per 100k Pop'

In [14]:
def add_pop_relative_stuff(kurzel):
    pop = shared.get_land_pop(kurzel)
    dfs[kurzel]['shots_sum_100k'] = dfs[kurzel]['shots_sum'] / pop * 100_000
    dfs[kurzel]['shots_today_100k'] = dfs[kurzel]['shots_today'] / pop * 100_000



for kurzel in bl_kurzel:
    add_pop_relative_stuff(kurzel)

## Let's take a look

In [15]:
dfs['DE'].head(15)

,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum,is_weekend,weekday_name,calendar_week,year_and_week,shots_sum_100k,shots_today_100k
0,2020-12-26,NaN,0.0,0.0,0.0,0.0,0,True,Sat,52,2020_52,0.0,0.0
1,2020-12-27,2020-12-28 16:15,"21,566.0","21,566.0",0.0,"21,566.0",21566,True,Sun,52,2020_52,25.9,25.9
2,2020-12-28,2020-12-29 08:00,"41,962.0","41,962.0",0.0,"20,396.0",41962,False,Mon,53,2020_53,50.4,24.5
3,2020-12-29,NaN,NaN,NaN,NaN,"44,832.0",86794,False,Tue,53,2020_53,104.3,53.9
4,2020-12-30,2020-12-31 08:30,"131,626.0","131,626.0",0.0,"44,832.0",131626,False,Wed,53,2020_53,158.2,53.9
5,2020-12-31,2021-01-01 12:30,"165,575.0","165,575.0",0.0,"33,949.0",165575,False,Thu,53,2020_53,199.0,40.8
6,2021-01-01,2021-01-02 08:00,"188,553.0","188,553.0",0.0,"22,978.0",188553,False,Fri,53,2020_53,226.7,27.6
7,2021-01-02,2021-01-03 08:00,"238,809.0","238,809.0",0.0,"50,256.0",238809,True,Sat,53,2020_53,287.1,60.4
8,2021-01-03,2021-01-04 12:00,"265,986.0","265,986.0",0.0,"27,177.0",265986,True,Sun,53,2020_53,319.7,32.7
9,2021-01-04,2021-01-05 12:00,"316,962.0","316,962.0",0.0,"50,976.0",316962,False,Mon,1,2021_01,381.0,61.3


In [16]:
dfs['DE'].tail(10)


,date,publication_date,dosen_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,shots_today,shots_sum,is_weekend,weekday_name,calendar_week,year_and_week,shots_sum_100k,shots_today_100k
374,2022-01-04,2022-01-05 08:00,"150,983,426.0","58,236,287.0","59,371,059.0","893,127.0",150983426,False,Tue,1,2022_01,"181,491.1","1,073.6"
375,2022-01-05,2022-01-06 08:00,"151,807,909.0","58,297,042.0","59,483,838.0","824,483.0",151807909,False,Wed,1,2022_01,"182,482.1",991.1
376,2022-01-06,2022-01-07 08:00,"152,496,126.0","58,351,202.0","59,574,879.0","688,217.0",152496126,False,Thu,1,2022_01,"183,309.4",827.3
377,2022-01-07,NaN,NaN,NaN,NaN,"467,714.0",152963840,False,Fri,1,2022_01,"183,871.6",562.2
378,2022-01-08,NaN,NaN,NaN,NaN,"467,714.0",153431554,True,Sat,1,2022_01,"184,433.9",562.2
379,2022-01-09,2022-01-10 08:00,"153,899,268.0","58,465,277.0","59,787,106.0","467,714.0",153899268,True,Sun,1,2022_01,"184,996.1",562.2
380,2022-01-10,2022-01-11 08:00,"154,550,464.0","58,516,607.0","59,883,814.0","651,196.0",154550464,False,Mon,2,2022_02,"185,778.9",782.8
381,2022-01-11,2022-01-12 08:00,"155,365,945.0","58,574,159.0","60,004,889.0","815,481.0",155365945,False,Tue,2,2022_02,"186,759.1",980.3
382,2022-01-12,2022-01-13 08:00,"156,257,670.0","58,639,456.0","60,141,697.0","891,725.0",156257670,False,Wed,2,2022_02,"187,831.0","1,071.9"
383,2022-01-13,2022-01-14 08:00,"157,130,393.0","58,701,417.0","60,272,356.0","872,723.0",157130393,False,Thu,2,2022_02,"188,880.1","1,049.1"


## Save to pickle

In [17]:
from datetime import datetime
print(f"{datetime.now()}")

for kurzel in bl_kurzel:
    path = f'data/df_vac_{kurzel}.pkl'
    dfs[kurzel].to_pickle(path)
    print(f'Saved pickle to {path}')


2022-01-14 19:04:12.151042
Saved pickle to data/df_vac_BB.pkl
Saved pickle to data/df_vac_BE.pkl
Saved pickle to data/df_vac_BW.pkl
Saved pickle to data/df_vac_BY.pkl
Saved pickle to data/df_vac_DE.pkl
Saved pickle to data/df_vac_HB.pkl
Saved pickle to data/df_vac_HE.pkl
Saved pickle to data/df_vac_HH.pkl
Saved pickle to data/df_vac_MV.pkl
Saved pickle to data/df_vac_NI.pkl
Saved pickle to data/df_vac_NW.pkl
Saved pickle to data/df_vac_RP.pkl
Saved pickle to data/df_vac_SH.pkl
Saved pickle to data/df_vac_SL.pkl
Saved pickle to data/df_vac_SN.pkl
Saved pickle to data/df_vac_ST.pkl
Saved pickle to data/df_vac_TH.pkl
